## Class
Has properties, methods. Define using 'class' keyword

#### Properties: 
- Class properties: Each instance of class will have same value
- instance properties: Each instance of class will have values what it is initialised with

#### Methods: 
- Instance methods: takes in self as first argument, needs no decorator.
- Class methods: takes in 'cls' (standard not must) as first arg. can be used to update class variables or as an alternate constructor.
- Static methods: normal python function but is relevant to class. does not depend on self or class object. 

In [39]:
import hashlib
class Employee():
    # global variables 
    # (_variablename is standard naming convention in python to define private variables)
    # ONLY NAMING NOT ENFORSED: You could still modify private variables in python
    
    _company = 'abc' 
    num_emp = 0

    # constructor
    def __init__(self, fname, lname, sal):
        self.fname = fname
        self.lname = lname
        self.sal = sal
        # using instance method to update class variable. 
        self.set_num_emp()         
        # we could directly use class object here and increment by 1
        # Employee.num_emp += 1 

    # instance method
    def print_emp(self):
        return f'company: {self._company}, full name: {self.fname} {self.lname}, sal: {self.sal}'

    # instance method to update class variable
    def set_num_emp(self):
        Employee.num_emp += 1

    # instance method defined as property. use @property decorator. can be accessed as a property. no () required. 
    # This is python's implementation of getter
    @property
    def fullname(self):
        return f'{self.fname} {self.lname}'

    
    @classmethod
    def from_string(cls, str): #cls is mandatory as first arg.
        fname, lname, pay = str.split('|')
        # return Employee(fname, lname, pay) - works but not advised. crazy things happen when we use inheritence
        return cls(fname, lname, pay)

    # normal python function to return a hash from string
    # can be made instance method as well. but make static if instance details are not required
    @staticmethod
    def get_hash(fullname):
        md5 = hashlib.md5()        
        md5.update(fullname.encode())        
        return md5.hexdigest()
        
        

## accessing class variables, static methods without instance creation

In [166]:
print(f'company name {Employee._company}')
print(f'num employees {Employee.num_emp}')
print(f'hash of company name is {Employee.get_hash(Employee._company)}')

company name Google
num employees 13
hash of company name is 8b36e9207c24c76e6719268e49201d94


## creating instance

In [167]:

emp1 = Employee('Jeff','B',100)

# accessing instance properties
print(emp1.fname)
print(emp1.lname)

# using instance methods. () is required
print(emp1.print_emp()) 

# using method as property. () not required
print(emp1.fullname)


Jeff
B
company: Google, full name: Jeff B, sal: 100
Jeff B


## using class method to create instance differently. alternate constructor

In [168]:
# using | separated string instead of fname, lname, pay
emp2 = Employee.from_string('Elon|M|200')
print(emp2.print_emp())
print(f'num employees {Employee.num_emp}')


company: Google, full name: Elon M, sal: 200
num employees 15


## modify class variables

In [169]:
# Modifying class variables using Class
Employee._company = 'Google'
emp3 = Employee('Sunder','P',300)
print(emp1.print_emp())
print(emp2.print_emp())
print(emp3.print_emp())
# Notice that company for emp1,emp2 also changed to google. reason (in print_emp, we are using self.company and we changed it to Google)


# Instead modify the class variable for specific instance
Employee.company = 'abc' # reverting to 'abc' 

emp1._company = 'Amazon'
emp2._company = 'Tesla'
emp3._company = 'Google'
print(emp1.print_emp())
print(emp2.print_emp())
print(emp3.print_emp())

company: Google, full name: Jeff B, sal: 100
company: Google, full name: Elon M, sal: 200
company: Google, full name: Sunder P, sal: 300
company: Amazon, full name: Jeff B, sal: 100
company: Tesla, full name: Elon M, sal: 200
company: Google, full name: Sunder P, sal: 300


## Getters, Setters, Deleters
- in our current class implementation, we could modify class variables, instance variables (both public + private) 
outside class implementation as we want. This should not be the case. 
- We should do necessary checks if the value to be updated is correct, do some validations. dont give access to private variables at all

## Getters: @property
## Setters: @property_name.setter
## Deleters: @property_name.deleter

In [170]:
import hashlib
class Employee2():
    def __init__(self, fname, lname, sal, company=None):
        self.fname = fname
        self.lname = lname
        self.sal = sal
        self._company = company if company else 'abc'

    def print_emp(self):
        return f'company: {self._company}, full name: {self.fname} {self.lname}, sal: {self.sal}'

    @property
    def company(self):
        return self._company

    @company.setter
    def company(self, value):
        if isinstance(value, str) and value: #checking if string, value is present
            self._company = value
        else:
            raise ValueError("Company must be a non-empty string")        

    @property
    def fullname(self):
        return f'{self.fname} {self.lname}'
    
    @classmethod
    def from_string(cls, str): #cls is mandatory as first arg.
        fname, lname, pay = str.split('|')
        return cls(fname, lname, pay)       
        

In [171]:
emp1 = Employee2('Jeff','B',100)
print(emp1.print_emp())

emp1.company = 'Amazon'
print(emp1.print_emp())


emp1.company = 1 ## fails as this doesn't pass setter validation



company: abc, full name: Jeff B, sal: 100
company: Amazon, full name: Jeff B, sal: 100


ValueError: Company must be a non-empty string

## Notes:
- notice that we moved company from class variable to instance variable. we can only use setter, getter in python on instance variables.
- first define getter using @property, use propertyname.setter decorator
- if we want to use company as class variable and still use getter, setter, then we have to chain
- create a new class with company as single instance variable, define getter, setter in that class, 
in current class user getter, setter from basecalss


In [172]:
class CompanyClass:
    def __init__(self, value):
        self.value = value

    @property
    def company(self):
        return self.value

    @company.setter
    def company(self, new_value):
        if isinstance(new_value, str) and new_value: #checking if string, value is present
            self.value = new_value
        else:
            raise ValueError("Company must be a non-empty string")      

    def __delete__(self, instance):
        raise AttributeError("Cannot delete attribute")

class Employee3:
    _company_class = CompanyClass('abc')
    def __init__(self, fname, lname, sal, company=None):
        self.fname = fname
        self.lname = lname
        self.sal = sal
        if company:
            self.company = company

    @property
    def company(self):
        return Employee3._company_class.company #using getter from base class (CompanyClass)

    @company.setter
    def company(self, new_value):
        Employee3._company_class.company = new_value #using setter from base class (CompanyClass)
    

    def print_emp(self):
        return f'company: {self.company}, full name: {self.fname} {self.lname}, sal: {self.sal}'

    @property
    def fullname(self):
        return f'{self.fname} {self.lname}'



In [173]:
emp1 = Employee3('Jeff','B',100)
print(emp1.print_emp())

emp1.company = 'Amazon'
print(emp1.print_emp())


emp1.company = '' # Fails as validation check fails. Notice that the setters are chained, we are getting the error from setter of base class
print(emp1.print_emp())


company: abc, full name: Jeff B, sal: 100
company: Amazon, full name: Jeff B, sal: 100


ValueError: Company must be a non-empty string

# Inheritance
- Mechanism to create a new class from an existing class, inheriting attributes and methods.

## Code Reusability:
- use existing class to create a new class. this new class will have access to all variables, methods of base class
- Eg: Manager is also an employee. Only additional info required for manager is list of reportees.

## Extensibility: (Extends)
- add new variables/ methods for base class (eg: reportee_list, add_reportee, remove_reportee. Manager has these but employee doesn't)

In [189]:
# Base class/ parent class
class Employee2():
    def __init__(self, fname, lname, sal, company=None):
        self.fname = fname
        self.lname = lname
        self.sal = sal
        self.company = company if company else 'abc'

    def print_emp(self):
        return f'company: {self._company}, full name: {self.fname} {self.lname}, sal: {self.sal}'

    @property
    def fullname(self):
        return f'{self.fname} {self.lname}'
    
    @classmethod
    def from_string(cls, str): #cls is mandatory as first arg.
        fname, lname, pay = str.split('|')
        return cls(fname, lname, pay)       

# "class Manager(Employee2)" in python is same as "class Manager extends Employee2" in Java
class Manager(Employee2): 
    def __init__(self, fname, lname, sal, company=None, reportee_list=None):
        super().__init__(fname, lname, sal, company)
        if reportee_list is None:
            reportee_list = []
        self.reportee_list = reportee_list
    
    def print_reportee(self):
        print(f'{self.fname} {self.lname}')
        for x in self.reportee_list:
            print(f'-- {x.fname} {x.lname}')

    def add_reportee(self, emp):
        if emp not in self.reportee_list:
            self.reportee_list.append(emp)

    def remove_reportee(self, emp):
        if emp in self.reportee_list:
            self.reportee_list.remove(emp)


In [190]:
emp1 = Employee2.from_string('Jeff|B|100')
emp2 = Employee2.from_string('Sundar|P|200')
emp3 = Employee2.from_string('Elon|M|300')

## We need to pass all the props required by base class + new props required by new class during instantiation
mgr1 = Manager('Bharath','T',1000,'IND',[emp1])
mgr1.print_reportee()
mgr1.add_reportee(emp2)
mgr1.add_reportee(emp3)
mgr1.print_reportee()
mgr1.remove_reportee(emp2)
mgr1.print_reportee()

# Note: We are not able directly edit fname, lname, pay, company of employee object if we use manager class

Bharath T
-- Jeff B
Bharath T
-- Jeff B
-- Sundar P
-- Elon M
Bharath T
-- Jeff B
-- Elon M


## polymorphism : 
- Polymorphism is the ability of different objects to be treated as instances of the same class through a common interface or base class.
- It allows methods to operate on objects of different classes as if they are of the same class.
- define layout, let all sub classes implement them separately. 
- we can use a single funtion to handle all base object calls (eg: speak) of diff derived classes (eg: cat, dog)
- ALL SUBCLASSES MUST HAVE ALL METHODS IMPLEMENTED IN BASE CLASS.

In [191]:
class Animal:
    def speak(self):
        raise NotImplementedError("Subclass must implement this method")

class Dog(Animal):
    def speak(self):
        return 'Bark'

class Cat(Animal):
    def speak(self):
        return 'Meow'

def make_animal_speak(animal):
    return animal.speak()

dog = Dog()
cat = Cat()
print(make_animal_speak(dog))
print(make_animal_speak(cat))

Bark
Meow


## specialization: 
- Creating a subclass that extends or modifies the behavior of a base class to cater to more specific needs.
- similar to polymorphism. Allows you to add custom behaviour for each subclass based on behaviour. 
- Eg: dog can bark, cat can purr

In [192]:
class Animal:
    def __init__(self, name):
        self.name = name

    def speak(self):
        raise NotImplementedError("Subclass must implement this method")

    def info(self):
        raise NotImplementedError("Subclass must implement this method")

class Dog(Animal):
    def speak(self):
        return 'Bark'

    def info(self):
        return f'I am a dog, name is {self.name}'

    def bark(self):
        return 'Bark'

class Cat(Animal):
    def speak(self):
        return 'Meow'

    def info(self):
        return f'I am a Cat, name is {self.name}'

    def meow(self):
        return 'Meow'


dog = Dog('Lucky')
cat = Cat('Malli')
print(dog.info())
print(dog.bark())


print(cat.info())
print(cat.meow())

I am a dog, name is Lucky
Bark
I am a Cat, name is Malli
Meow


- Eg2: Circle, rectangle extends share class, their area is calculated differently

In [193]:
class Shape():
    def area():
        raise NotImplementedError("Subclass must implement this method")

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14 * self.radius * self.radius

class Rectangle(Shape):
    def __init__(self, length, breadth):
        self.length = length
        self.breadth = breadth

    def area(self):
        return self.length * self.breadth


circle = Circle(1)
rectangle = Rectangle(1,2)

print(circle.area())
print(rectangle.area())

3.14
2


## abstraction
- Hiding the complex implementation details and showing only the necessary features of an object.
- Similar to specialization
- additional import, @abstractmethod decorater

In [ ]:
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

    @abstractmethod
    def perimeter(self):
        pass

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

    def perimeter(self):
        return 2 * (self.width + self.height)

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14 * self.radius * self.radius

    def perimeter(self):
        return 2 * 3.14 * self.radius

# Usage
shapes = [Rectangle(2, 3), Circle(5)]
for shape in shapes:
    print(f"Area: {shape.area()}, Perimeter: {shape.perimeter()}")


#### Java Example

In [ ]:
abstract class Employee {
    String name;
    int id;

    Employee(String name, int id) {
        this.name = name;
        this.id = id;
    }

    abstract double calculatePay();
}

class SalariedEmployee extends Employee {
    double salary;

    SalariedEmployee(String name, int id, double salary) {
        super(name, id);
        this.salary = salary;
    }

    @Override
    double calculatePay() {
        return salary;
    }
}

class HourlyEmployee extends Employee {
    double hourlyRate;
    int hoursWorked;

    HourlyEmployee(String name, int id, double hourlyRate, int hoursWorked) {
        super(name, id);
        this.hourlyRate = hourlyRate;
        this.hoursWorked = hoursWorked;
    }

    @Override
    double calculatePay() {
        return hourlyRate * hoursWorked;
    }
}


## interface, implements
- Interface: A contract that defines a set of methods without implementing them.
- It specifies "what" a class should do but not "how" it should do it.
- Implements: Keyword used in some languages to denote that a class is implementing an interface.
- Python does not have a built-in interface keyword. interfaces can be implemented using abstract base classes (ABCs). (similar to above example)

In [195]:
from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def speak(self):
        pass

    @abstractmethod
    def move(self):
        pass

class Dog(Animal):
    def speak(self):
        return "Bark"

    def move(self):
        return "Run"

class Cat(Animal):
    def speak(self):
        return "Meow"

    def move(self):
        return "Walk"

# Usage
animals = [Dog(), Cat()]
for animal in animals:
    print(f"Speak: {animal.speak()}, Move: {animal.move()}")


Speak: Bark, Move: Run
Speak: Meow, Move: Walk


#### Java Example

In [ ]:
interface Flyable {
    void fly();
}

interface Swimmable {
    void swim();
}

class Duck implements Flyable, Swimmable {
    @Override
    public void fly() {
        System.out.println("Duck is flying");
    }

    @Override
    public void swim() {
        System.out.println("Duck is swimming");
    }
}


## encapsulation
- Bundling the data and methods that operate on the data into a single unit, typically a class, and restricting access to some of the object's components.


In [ ]:
class Person:
    def __init__(self, name, age):
        self.__name = name  # private attribute
        self.__age = age    # private attribute

    def get_name(self):
        return self.__name

    def set_name(self, name):
        self.__name = name

    def get_age(self):
        return self.__age

    def set_age(self, age):
        if age > 0:
            self.__age = age
        else:
            raise ValueError("Age must be positive")

# Usage
person = Person("Alice", 30)
print(person.get_name())  # Output: Alice
person.set_age(31)
print(person.get_age())   # Output: 31


## composition
- A design principle where a class is composed of one or more objects from other classes, rather than inheriting from them.
- Car class has a engine class, suspensin class, braking class, ...

In [197]:
class Engine:
    def start(self):
        return "Engine started"

class Brake:
    def apply_brake(self):
        return "Brake applied"

class Car:
    def __init__(self):
        self.engine = Engine()  # Car has an Engine
        self.brake = Brake()

    def start(self):
        return self.engine.start()

    def slow_down(self):
        return self.brake.apply_brake()
# Usage
car = Car()
print(car.start())  
print(car.slow_down())  

Engine started
Brake applied


## Aggregation
- A form of composition where a class is composed of objects that can exist independently of the parent class.

In [198]:
class TeamMember:
    def __init__(self, name):
        self.name = name

class Team:
    def __init__(self):
        self.members = []

    def add_member(self, member):
        self.members.append(member)

    def get_members(self):
        return [member.name for member in self.members]

# Usage
member1 = TeamMember("Alice")
member2 = TeamMember("Bob")
team = Team()
team.add_member(member1)
team.add_member(member2)
print(team.get_members())  


['Alice', 'Bob']


## Association
- A relationship between two classes that establishes a connection between their objects.

In [199]:
class Teacher:
    def __init__(self, name):
        self.name = name

class Course:
    def __init__(self, title):
        self.title = title
        self.teacher = None

    def assign_teacher(self, teacher):
        self.teacher = teacher

# Usage
teacher = Teacher("Mr. Smith")
course = Course("Math")
course.assign_teacher(teacher)
print(course.teacher.name)  


Mr. Smith


## Dependency Injection
- A design pattern in which an object receives other objects that it depends on, rather than creating them itself.

In [200]:
from abc import ABC, abstractmethod

# Step 1: Define the abstract base class for the database connector
class DatabaseConnector(ABC):
    @abstractmethod
    def connect(self):
        pass

    @abstractmethod
    def execute_query(self, query):
        pass

# Step 2: Implement the MySQL connector
class MySQLConnector(DatabaseConnector):
    def connect(self):
        return "Connected to MySQL"

    def execute_query(self, query):
        return f"Executing '{query}' on MySQL"

# Step 3: Implement the Oracle connector
class OracleConnector(DatabaseConnector):
    def connect(self):
        return "Connected to Oracle"

    def execute_query(self, query):
        return f"Executing '{query}' on Oracle"

# Step 4: Implement the PostgreSQL connector
class PostgresConnector(DatabaseConnector):
    def connect(self):
        return "Connected to PostgreSQL"

    def execute_query(self, query):
        return f"Executing '{query}' on PostgreSQL"

# Step 5: Define the DatabaseService class which uses dependency injection
class DatabaseService:
    def __init__(self, db_connector: DatabaseConnector):
        self.db_connector = db_connector
        print(self.db_connector.connect())

    def run_query(self, query):
        return self.db_connector.execute_query(query)

# Step 6: Usage
# Choose a connector and inject it into the DatabaseService
mysql_connector = MySQLConnector()
oracle_connector = OracleConnector()
postgres_connector = PostgresConnector()

# Create a database service with the desired connector
db_service_mysql = DatabaseService(mysql_connector)
print(db_service_mysql.run_query("SELECT * FROM users"))

db_service_oracle = DatabaseService(oracle_connector)
print(db_service_oracle.run_query("SELECT * FROM employees"))

db_service_postgres = DatabaseService(postgres_connector)
print(db_service_postgres.run_query("SELECT * FROM products"))


Connected to MySQL
Executing 'SELECT * FROM users' on MySQL
Connected to Oracle
Executing 'SELECT * FROM employees' on Oracle
Connected to PostgreSQL
Executing 'SELECT * FROM products' on PostgreSQL
